# 8.4 Backend Code

In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
import pandas as pd
!pip install pyngrok
import numpy as np
!pip install flask_cors
from flask_cors import CORS
from pyngrok import ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19879 sha256=62e3f875c66af613131552751d4cf6013532e06bfdf6b4233c3eb96fc352a4f7
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

os.chdir('drive/MyDrive/Major Project/')
path = ''

In [ ]:
model = tf.keras.models.load_model('my_resnet50_model.h5')


In [ ]:
!ngrok config add-authtoken 2Ka4LuEh8lYla14AW6DjZZCsxdZ_6Lo3Ke8iP9yRQvHYS3PHa
https_tunnel = ngrok.connect(bind_tls=True)

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
app = Flask(__name__)
CORS(app)
@app.route('/')
def hello():
    return 'This is a flask server hosting transfer learning model for classification of Arrhythmia'

@app.route('/predict', methods=['POST'])
def predict():
    # Get the patient's CSV file from the request
    patient_csv = request.files['file']
    print(patient_csv)
    # Load the CSV file back into a NumPy array
    loaded_array = np.genfromtxt(patient_csv, delimiter=',')
    print(loaded_array.shape)
    # Reshape the loaded array to match the original shape
    reloaded_array = loaded_array.reshape((1, 360, 1))
    print(reloaded_array.shape)
    # Predict the arrhythmia class using the ResNet50 model
    prediction = model.predict(reloaded_array)
    print(prediction)
    true_prediction=[]
    for element in prediction:
        true_prediction.append(np.argmax(element))
    print(true_prediction)
    print('///////////////////////////////////////////////////////////')

    # Convert the prediction to a class label
    arrhythmia_class=""
    if true_prediction==[0]:
      arrhythmia_class="N: Normal beat (normal rhythm of the heart)"
    elif true_prediction==[1]:
      arrhythmia_class="S: Supraventricular premature beat (an early beat that arises from the atria or the AV node)"
    elif true_prediction==[2]:
      arrhythmia_class="F: Fusion of ventricular and normal beat (a beat that has characteristics of both a normal beat and a ventricular premature beat)"
    elif true_prediction==[3]:
      arrhythmia_class="V: Premature ventricular contraction (an early beat that arises from the ventricles)"
    elif true_prediction==[4]:
      arrhythmia_class="Q: Unclassifiable beat (a beat that cannot be classified as any of the above categories)"




    # Return the predicted class label
    return jsonify({"arrhythmia class": arrhythmia_class})

In [ ]:

if __name__ == '__main__':
    app_url = ngrok.connect(addr=80, proto='http')
    print(f'Flask app running at: {app_url}')
    app.run(host='localhost', port=80)


Flask app running at: NgrokTunnel: "https://2a7a-34-80-226-243.ngrok-free.app" -> "http://localhost:80"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:80
INFO:werkzeug:Press CTRL+C to quit


<FileStorage: 'test_patient_1.csv' ('application/octet-stream')>
(360,)
(1, 360, 1)
1/1 [==============================] - 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [06/May/2023 04:34:31] "POST /predict HTTP/1.1" 200 -


[[9.0374672e-01 6.1314440e-06 1.2609958e-04 4.9952581e-05 9.6071035e-02]]
[0]
///////////////////////////////////////////////////////////
<FileStorage: 'test_patient_2.csv' ('application/octet-stream')>
(360,)
(1, 360, 1)
1/1 [==============================] - 0s 52ms/step


INFO:werkzeug:127.0.0.1 - - [06/May/2023 04:34:44] "POST /predict HTTP/1.1" 200 -


[[9.0478712e-01 8.8747092e-06 3.0292021e-04 8.1549464e-05 9.4819605e-02]]
[0]
///////////////////////////////////////////////////////////
